In [ ]:
import pandas as pd
import openpyxl
import json
import os
import re

input_json_file = 'sample/en.json'
out_excel_file = 'hi/hi_edited.xlsx'
# translation_excel_file = ['sample/sample.xlsx']

In [ ]:
# this finds our json files
path_to_json = './hi'
translation_excel_file = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.xlsx')]


In [ ]:
with open(input_json_file) as f:
    js = json.load(f)

In [ ]:
out_df = pd.DataFrame(list(js.items()),
                   columns=['Key', 'To be Translated'])

In [ ]:
out_df['Key']

In [ ]:
out_df["Key"].to_excel('tran.xlsx')

In [ ]:
lang = 'Hindi'
english_col = 'English copy'
allowed_values = ['x','y','z']

In [ ]:
excelDf = pd.DataFrame([], columns=['Key', lang])
for excel_file_name in translation_excel_file:
    print(excel_file_name)
    excel = pd.ExcelFile('hi/'+excel_file_name)
    
    count = 0
    for sheetName in excel.sheet_names:
        sheet = excel.parse(sheet_name = sheetName, header=1)
        print(sheet.columns)
        FORMAT = [english_col,lang]
        for value in allowed_values:
            if value in sheet.columns:
                FORMAT.append(value)
        filteredSheet = sheet[FORMAT]
        sheet_no_na = filteredSheet.dropna(subset = [english_col, lang], inplace=False)
        sheet_new = sheet_no_na.rename(columns = {english_col: 'Key'}, inplace=False)
        count += sheet_new.count()
        excelDf = pd.concat([excelDf, sheet_new], axis=0)

In [ ]:
excelDf['Key_lower'] = excelDf['Key'].str.lower()
out_df['Key_lower'] = out_df['Key'].str.lower()

In [ ]:
excelDf_dropped = excelDf.drop_duplicates(subset=['Key_lower'], keep='first')
out_df_dropped = out_df.drop_duplicates(subset=['Key'], keep='first')

In [ ]:
df_diff = pd.merge(out_df_dropped, excelDf_dropped, on="Key_lower",how='left')

In [ ]:
df_diff = df_diff.drop_duplicates(subset=['Key_x'], keep='first')

In [ ]:
df_diff = df_diff[['Key_x', 'To be Translated', lang]]

In [ ]:
df_diff_a = df_diff[df_diff[lang].isna()]

In [ ]:
df_diff[df_diff[lang].isna()].count()

In [ ]:
df_diff_a = df_diff_a.rename(columns = {'Key_x': 'Key'}, inplace=False)

In [ ]:
df_diff_a.to_excel(out_excel_file)

In [ ]:
excelDf = pd.DataFrame([], columns=['Key', lang])
for excel_file_name in translation_excel_file:
    excel = pd.ExcelFile('hi/'+excel_file_name)
    print(excel_file_name)
    count = 0
    for sheetName in excel.sheet_names:
        sheet = excel.parse(sheet_name = sheetName, header=1)
        FORMAT = [english_col,lang]
        print(sheet.columns)
        for value in allowed_values:
            if value in sheet.columns:
                FORMAT.append(value)
        filteredSheet = sheet[FORMAT]
        sheet_no_na = filteredSheet.dropna(subset = [english_col, lang], inplace=False)
        sheet_new = sheet_no_na.rename(columns = {english_col: 'Key'}, inplace=False)
        count += sheet_new.count()
        excelDf = pd.concat([excelDf, sheet_new], axis=0)

In [ ]:
excelDf.to_excel('allHindi.xlsx')

In [ ]:
def getKeysWithTranslation(lang):
    with open(lang+"/"+lang+".json") as f:
        js = json.load(f)
    
    keys_list = []
    for key, value in js.items():
        if key != value:
            keys_list.append(key)
    return keys_list
    

In [ ]:
getKeysWithTranslation('hi')

In [ ]:
##--------- NEW CODE FOR TRANSLATION GENERATION ------

In [1]:
import pandas as pd
import openpyxl
import json
import os
import re
from pandas import ExcelWriter

In [2]:

def findKeysWithoutTranslation(lang):
    with open('./../../../crowdsource-ui/locales/{lang}.json'.format(lang=lang)) as f:
        js = json.load(f)
    
    keys_without_translation = []
    for key, value in js.items():
        if key == value and value :
            keys_without_translation.append(key)
    return keys_without_translation

In [9]:
def extract_tags(text):
    allowed_replacements = ["x", "y", "z", "u","v","w", "a-tag-replacement"]
    regex = r"<(\S*?)[^>]*>.*?<\/\1>|<.*?\/>"
    out_txt = text
    matches = re.finditer(regex, out_txt, re.MULTILINE)
    index = 0
    matched_tags_dict = {}
    for match in matches:
        matched_tag = match.group()
        if "<b>" in matched_tag:
            continue
        if "<a" in matched_tag:
            attrs = matched_tag[matched_tag.find('<a')+2: matched_tag.find('>')]
            matched_tag_replacement = matched_tag.replace(attrs,"")
            out_txt = out_txt.replace(matched_tag, matched_tag_replacement)
            matched_tags_dict['a-tag-replacement'] = attrs
        else:
            replacement = allowed_replacements[index]
            out_txt = out_txt.replace(matched_tag, '<{}>'.format(replacement))
            index+=1
            matched_tags_dict[replacement] = matched_tag
    return out_txt , matched_tags_dict

In [5]:
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for lang_code, lang_df in list_dfs.items():
            lang_df.to_excel(writer,lang_code)
        writer.save()
    print('done')

In [4]:
def get_text_from_html_tags(tag):
    regex = r"<(\S*?)[^>]*>(.*?)<\/\1>"
    out_tag = tag
    matches = re.finditer(regex, out_tag, re.MULTILINE)
    string_matches = set()
    for match in matches:
        match_group_length = len(match.groups())
        if match_group_length>1 and len(match.group(2)) != 0:
            string_matches.add(match.group(2).strip())
    return string_matches

In [6]:
def get_dict_for_data(text, extracted_tags):
    out_dict = {}
    out_dict["English copy"] = [text]
    
    if len(extracted_tags) == 0:
        out_dict["Comments"] = [""]
    else:
        out_dict["Comments"] = [text]
        
    for replacement in sorted (extracted_tags.keys()):
        out_dict[replacement] = [extracted_tags[replacement]]
    return out_dict

In [7]:
languages = [('hi', "Hindi"),('gu', "Gujarati"),('as', "Assamese"),('bn','Bengali'),('ta',"Tamil"),
             ('te',"Telugu"),('mr',"Marathi"),('pa',"Punjabi"),('ml',"Malayalam"),('or',"Odia"),('kn',"Kannada")]
all_dfs = {}
for lang_code, lang_name in languages:
    lang_df = pd.DataFrame([], columns=[])
    keys_without_translation = findKeysWithoutTranslation(lang_code)
    for key in keys_without_translation:
        processed_text, extracted_tags = extract_tags(key)
    
        data_dict = get_dict_for_data(processed_text, extracted_tags)
    
        try:
            tmp_df = pd.DataFrame.from_dict(data_dict, orient='columns')
            lang_df = lang_df.append(tmp_df, ignore_index=True)
        except Exception as e:
            print(e, "\n", data_dict, "\n\n")
    lang_df[lang_name] = ""
    all_dfs[lang_code] = lang_df

In [8]:
save_xls(all_dfs, 'final1.xls')

done


/Users/nireshkumarr/miniconda3/envs/translation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# pd.DataFrame.from_dict(dict([('A', [1, 2, 3]), ('B', [4, 5, 6])]),orient='columns')

dct = {"English copy":["ABCD"], "x":["<>"], "y":["<>"],"a":["dsfs"]}
pd.DataFrame.from_dict(dct,orient='columns')
